<a href="https://colab.research.google.com/github/venki05/Mavericks_oneAPI_hack_kpr/blob/main/invoice%20processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Pdf to png converter
import fitz  # PyMuPDF
import os

def pdf_to_png(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all PDF files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(input_folder, filename)
            pdf_document = fitz.open(pdf_path)
            pdf_name = os.path.splitext(filename)[0]

            # Convert each page in the PDF to PNG
            for page_num in range(len(pdf_document)):
                page = pdf_document.load_page(page_num)
                pix = page.get_pixmap()

                # Save each page as a PNG in the output folder, with a unique name
                output_path = os.path.join(output_folder, f"{pdf_name}page{page_num + 1}.png")
                pix.save(output_path)
                print(f"Saved {output_path}")

            pdf_document.close()

# Specify the input and output folders
input_folder = "C:\\Users\\MURALI\\OneDrive - SSN-Institute\\Documents\\Invoices"
output_folder = "C:\\Users\\MURALI\\OneDrive - SSN-Institute\\Documents\\Output PNGs"

# Run the conversion function
pdf_to_png(input_folder, output_folder)

In [ ]:
# Main rogram
from PIL import Image
import pytesseract
import re
import pandas as pd
import os

# Path to the Tesseract executable (set the correct path on your system)
# Uncomment the line below if you haven't set tesseract in your system path.
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_invoice_details(image_path):
    # Load the image
    img = Image.open(image_path)

    # Extract text from the image using Tesseract OCR
    extracted_text = pytesseract.image_to_string(img)

    # Initialize a dictionary to store extracted fields
    invoice_data = {}

    # Define patterns to extract required information
    invoice_number_pattern = r"#\s*(IN\d{4}-\d{2}/\d+)"
    due_date_pattern = r"Due Date:?\s*(\d{2}-[A-Z]{3}-\d{4})"
    invoice_date_pattern = r"Invoice Date:?\s*(\d{2}-[A-Z]{3}-\d{4})"
    total_amount_pattern = r"Total\sAmt\s*([\d,]+)"
    supplier_pattern = r"SSN College of Engineering"

    # Extract Invoice Number
    invoice_match = re.search(invoice_number_pattern, extracted_text)
    invoice_data['Invoice Number'] = invoice_match.group(1) if invoice_match else None

    # Extract Due Date
    due_date_match = re.search(due_date_pattern, extracted_text)
    invoice_data['Due Date'] = due_date_match.group(1) if due_date_match else None

    # Extract Invoice Date
    invoice_date_match = re.search(invoice_date_pattern, extracted_text)
    invoice_data['Invoice Date'] = invoice_date_match.group(1) if invoice_date_match else None

    # Extract Total Amount
    total_amount_match = re.search(total_amount_pattern, extracted_text)
    invoice_data['Total Amount'] = total_amount_match.group(1) if total_amount_match else None

    # Extract Supplier Name (which is constant here)
    supplier_match = re.search(supplier_pattern, extracted_text)
    invoice_data['Supplier'] = "SSN College of Engineering" if supplier_match else None

    return invoice_data

def process_images_to_excel(input_folder, output_excel):
    # List to hold all extracted data
    all_data = []

    # Iterate over each PNG file in the folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            invoice_details = extract_invoice_details(image_path)
            all_data.append(invoice_details)

    # Convert all collected data to a DataFrame
    df = pd.DataFrame(all_data)


# Save the DataFrame to an Excel file
    df.to_excel(output_excel, index=False)
    print(f"All invoice details have been saved to {output_excel}")

# Specify the folder containing PNG files and the output Excel file path
input_folder = r"C:\Users\MURALI\OneDrive - SSN-Institute\Documents\Output PNGs"
output_excel = r"C:\Users\MURALI\OneDrive - SSN-Institute\Documents\Project.xlsx"

# Process the images and save to Excel
process_images_to_excel(input_folder, output_excel)
